<a href="https://colab.research.google.com/github/byui-cse/cse380-notebooks/blob/master/09_2_Ponder_and_Prove_Data_Compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ponder and Prove Data Compression
#### Due: Saturday, 6 March 2021, 11:59 pm.

# TODO Explore Huffman Trees and Huffman Codes


Your task is examine how to compress a *special piece of information* as compactly as possible, and **calculate various compression ratios**.

Recall that the **compression ratio** of a variable-length encoding like Huffman encoding is the percentage $100(f - v)/f$, where $f$ is the number of bits per symbol of the smallest **fixed**-length encoding, and $v$ is the average number of bits per symbol with the variable-length encoding.

For example, if there were 9 different symbols in a message, $f=4$ is the number of bits of the smallest fixed-length encoding, because $2^3 = 8$ (not enough for $9$) and $2^4 = 16$ (enough and to spare). If the variable-length encoding of the message had $v=3.12$, the compression ratio would be $100(4 - 3.12)/4 \approx 22\%$.

Note that calculating the average number of bits per symbol is not strictly necessary. That's because an alternate and equivalent way is to calculate $100(ft - vt)/ft$, where $ft$ is the **total** number of bits encoded with the fixed encoding, and $vt$ is the **total** number of bits encoded with the variable-length encoding.

The *special piece of information* to be compressed is a list of the first ten million primes. This is a list that starts

|    |
|----|
|  2 |
|  3 |
|  5 |
|  7 |
| 11 |
| 13 |
| 17 |
| 19 |
| 23 |
| 29 |

  and ends

|           |
|-----------|
| 179424551 |
| 179424571 |
| 179424577 |
| 179424601 |
| 179424611 |
| 179424617 |
| 179424629 |
| 179424667 |
| 179424671 |
| 179424673 |

As ASCII text stored in a file with one prime per line, the size of this data file is slightly over 89 megabytes. The goal is to compress this down to just over 5 megabytes (5589056 bytes, to be exact). That's a 94% compression ratio!

Standard compression tools can only get about a 73% compression ratio for this ASCII data. A more clever approach is needed. Instead of compressing the list of prime numbers, compress a list of the *gaps* between them!

It doesn't save much, just the unique (occurring only once) gap size of 1 between 2 and 3, but in the spirit of de Polignac's conjecture that every *even* number appears infinitely often as a gap between consecutive primes, just consider the even-sized gaps. The result will be a list that starts with 2 (the difference between 5 and 3), 2 (the difference between 7 and 5), 4 (the difference between 11 and 7), 2 (the difference between 13 and 11), 4 (the difference between 17 and 13), 2 (the difference between 19 and 17), 4 (the difference between 23 and 19), and 6 (the difference between 29 and 23).

Generating this data is the first task. The algorithm for doing so is very straightforward:

1. Find the gaps between consecutive odd primes.
2. Store these gaps as a list of even numbers.

Tabulating the results, the first ten gaps and the last ten gaps are as follows, where the numbers after the equals signs are the gaps to list:

|                 |
|-----------------|
|  5  -   3  =  2 |
|  7  -   5  =  2 |
| 11  -   7  =  4 |
| 13  -  11  =  2 |
| 17  -  13  =  4 |
| 19  -  17  =  2 |
| 23  -  19  =  4 |
| 29  -  23  =  6 |
| 31  -  29  =  2 |
| 37  -  31  =  6 |

|                                |
|--------------------------------|
| 179424551  -  179424533  =  18 |
| 179424571  -  179424551  =  20 |
| 179424577  -  179424571  =   6 |
| 179424601  -  179424577  =  24 |
| 179424611  -  179424601  =  10 |
| 179424617  -  179424611  =   6 |
| 179424629  -  179424617  =  12 |
| 179424667  -  179424629  =  38 |
| 179424671  -  179424667  =   4 |
| 179424673  -  179424671  =   2 |

As a correctness check, see if your generated list of gaps has length 9999998.

The next step is to count how many times each gap size occurs, so that for the Huffman encoding scheme, the larger the frequency of occurrence, the smaller the number of bits encoding that gap size.

As a correctness check, here are the first ten and the last ten gap counts:

|  Gap | Count   |
|------|---------|
|    2 |  738597 |
|    4 |  738717 |
|    6 | 1297540 |
|    8 |  566151 |
|   10 |  729808 |
|   12 |  920661 |
|   14 |  503524 |
|   16 |  371677 |
|   18 |  667734 |
|   20 |  354267 |
|      |         |
|  190 |       1 |
|  192 |       3 |
|  194 |       1 |
|  196 |       1 |
|  198 |       6 |
|  202 |       2 |
|  204 |       3 |
|  210 |       4 |
|  220 |       1 |
|  222 |       1 |

Note two things from these partial gap counts:

1. Small even numbers (< 100) are well represented, larger ones (< 1000) less so.
2. Ten million primes aren't enough to have *every* even number represented; for example, 200, 206, 208, 212, 214, 216, and 218 do not appear even once.


In [12]:

from primesieve import *
list_of_gaps = []
prev = 3
gap = 0
for prime in list(primes(4, 179424674)): #millionth prime 15485867
    gap = prime - prev
    prev = prime
    list_of_gaps.append(gap)
print(len(list_of_gaps))


9999998


In [5]:
from math import ceil, log
from collections import Counter

def show_results(message, code_tuples):
  total_characters = len(message)
  total_unique_characters = len(code_tuples)
  total_bits = 0
  for char, count, code in code_tuples:
    total_bits += count * len(code)
  average_bits_per_character = total_bits / total_characters
  fixed_bits_per_character = ceil(log(total_unique_characters, 2))
  total_fixed_bits = total_characters * fixed_bits_per_character
  compression_ratio = (total_fixed_bits - total_bits) / total_fixed_bits
  print(f'          Total Characters: {total_characters}')
  print(f'   Total Unique Characters: {total_unique_characters}')
  print(f'                Total Bits: {total_bits}')
  print(f'Average Bits per Character: {average_bits_per_character:.2f}')
  print(f'  Fixed Bits per Character: {fixed_bits_per_character}')
  print(f'          Total Fixed Bits: {total_fixed_bits}')
  print(f'         Compression Ratio: {compression_ratio:.3f}')

In [13]:
prime_tuple = \
[('2', 738597, '0'),
 ('4', 738717, '1'),
 ('6', 1297540, '10'),
 ('8', 566151, '11'),
 ('10',	729808, '100'),
 ('12',	920661, '101'),
 ('14',	503524, '110'),
 ('16',	371677, '111'),
 ('18',	667734, '1000'),
 ('20',	354267, '1001'),
 ('22', 307230, '1010'),
 ('24', 453215, '1011'),
 ('26', 211203, '1100'),
 ('28', 229177, '1101'),
 ('30', 398713, '1110'),
 ('32', 123123, '1111'),
 ('34', 129043, '10000'),
 ('36', 206722, '10001'),
 ('38', 94682, '10010'),
 ('40', 111546, '10011'),
 ('42', 159956, '10100'),
 ('44', 64866, '10101'),
 ('46', 54931, '10110'),
 ('48', 93693, '10111'),
 ('50', 52183, '11000'),
 ('52', 38800, '11001'),
 ('54', 64157, '11010'),
 ('56', 32224, '11011'),
 ('58', 27985, '11100'),
 ('60', 55305, '11101'),
 ('62', 16763, '11110'),
 ('64', 17374, '11111'),
 ('66', 30960, '100000'),
 ('68', 12368, '100001'),
 ('70', 17475, '100010'),
 ('72', 17255, '100011'),
 ('74', 8540, '100100'),
 ('76', 7253, '100101'),
 ('78', 13758, '100110'),
 ('80', 6760, '100111'),
 ('82', 4791, '101000'),
 ('84', 9818, '101001'),
 ('86', 3411, '101010'),
 ('88', 3454, '101011'),
 ('90', 7056, '101100'),
 ('92', 2259, '101101'),
 ('94', 2058, '101110'),
 ('96', 3544, '101111'),
 ('98', 1831, '110000'),
 ('100', 1923, '110001'),
 ('102', 2374, '110010'),
 ('104', 1168, '110011'),
 ('106', 933, '110100'),
 ('108', 1634, '110101'),
 ('110', 941, '110110'),
 ('112', 711, '110111'),
 ('114', 1125, '111000'),
 ('116', 439, '111001'),
 ('118', 433, '111010'),
 ('120', 948, '111011'),
 ('122', 287, '111100'),
 ('124', 318, '111101'),
 ('126', 533, '111110'),
 ('128', 183, '111111'),
 ('130', 211, '1000000'),
 ('132', 301, '1000001'),
 ('134', 128, '1000010'),
 ('136', 100, '1000011'),
 ('138', 210, '1000100'),
 ('140', 140, '1000101'),
 ('142', 90, '1000110'),
 ('144', 123, '1000111'),
 ('146', 46, '1001000'),
 ('148', 67, '1001001'),
 ('150', 94, '1001010'),
 ('152', 52, '1001011'),
 ('154', 43, '1001100'),
 ('156', 57, '1001101'),
 ('158', 19, '1001110'),
 ('160', 27, '1001111'),
 ('162', 27, '1010000'),
 ('164', 20, '1010001'),
 ('166', 9, '1010010'),
 ('168', 25, '1010011'),
 ('170', 18, '1010100'),
 ('172', 4, '1010101'),
 ('174', 10, '1010110'),
 ('176', 11, '1010111'),
 ('178', 12, '1011000'),
 ('180', 10, '1011001'),
 ('182', 5, '1011010'),
 ('184', 4, '1011011'),
 ('186', 3, '1011100'),
 ('188', 1, '1011101'),
 ('190', 1, '1011110'),
 ('192', 3, '1011111'),
 ('194', 1, '1100000'),
 ('196', 1, '1100001'),
 ('198', 6, '1100010'),
 ('202', 2, '1100011'),
 ('204', 3, '1100100'),
 ('210', 4, '1100101'),
 ('220', 1, '1100110'),
 ('222', 1, '1100111')]
show_results(list_of_gaps, prime_tuple)

          Total Characters: 9999998
   Total Unique Characters: 104
                Total Bits: 30873980
Average Bits per Character: 3.09
  Fixed Bits per Character: 7
          Total Fixed Bits: 69999986
         Compression Ratio: 0.559


# TODO Determine Exact Size of Data to be Compressed


Without actually doing it, imagine creating an ASCII file containing the first ten million primes, represented in decimal, one prime per line. Calculate the size of this file, so you can show an exceptional compression ratio from it (see below).

Using a binary encoding instead of ASCII, each prime requires 32 bits (4 bytes), so the size of a binary file is easily determined.

Using a fixed-width encoding of the gap counts, however, requires knowing how many different gap sizes there are, after which the calculation is straightforward.

# TODO Use Functional Python


You are encouraged to use the [anytree](https://pypi.org/project/anytree) Python library, which has a nice exporter by way of which you can graphically view trees. (You may recall using this in DM1, and thus know that **anytree** depends on [graphviz](https://graphviz.org), which you also used.)

This library uses the object-oriented features of Python to create and visualize trees. You are encouraged to use the functional features of Python as much as possible, achieving your results not by using some existing third-party libraries for building Huffman Trees and Codes, but writing your own code as cleanly and elegantly as you can.

# TODO Achieve Target Compression Ratios

Your solution should correctly compute the following three compression ratios:

| Ratio       | Value              |
|-------------|--------------------|
| From fixed  | 36.125168653605158 |
| From binary |              86.03 |
| From ASCII  |              94.02 | 


# TODO My Report on What I Did and What I Learned

## Fun


I had a lot of fun exploring huffman trees even if I didn't get it right. It was really fun to see how much faster primesieve was than primerange. I may have even done things the hard way, but that was fun too to figure out the calculations and come up with code to help me on my way. 

## New

I learned how to use anytree. It is useful in visualizing how huffman trees work. I much more prefer to see a tree of the data than just staring at code that doesn't fully make sense to me. 

## Meaningful


I think I can really build upon using primes and creating lists and ranges with those primes and primesieve. I know that I can generate a large list and find gaps to help compress that list a little. This may be useful in future assignments like time saving and comprehension. For example, this assignment has helped me to know better how to iterate through a list. I have been a bit slow on understanding that with python. 

## Other

I don't feel impressed to add anything at this time. I had no collaberators, not for lack of trying. 

# TODO What is True?
Click on each warranted checkbox to toggle it to True (or back to False). 

NOTE: *This only works in Colab. If you run it in some other Jupyter notebook client/server environment you may have to change False to True (or vice versa) manually.*

This self-assessment is subject to revision by a grader.

In [14]:
#@markdown ## What is True about what I did?
#@markdown ### I had fun.
cb00 = True #@param {type:'boolean'}
#@markdown ### I learned something new.
cb01 = True #@param {type:'boolean'}
#@markdown ### I achieved something meaningful, or something I can build upon at a later time.
cb02 = True #@param {type:'boolean'}
#@markdown ## What is True about my report?
#@markdown ### I wrote a sufficient number of well-written sentences.
cb03 = True #@param {type:'boolean'}
#@markdown ### My report is free of mechanical infelicities.
cb04 = True #@param {type:'boolean'}
#@markdown ### I used Grammarly (or something better described in my report) to check for MIs.
cb05 = True #@param {type:'boolean'}
#@markdown ### I reported on any connections I found between these problems and something I already know. 
cb06 = True #@param {type:'boolean'}
#@markdown ### I reported who were and what contribution each of my collaborators made.
cb07 = True #@param {type:'boolean'}
#@markdown ## What is True about my calculations?
#@markdown ### I correctly calculated the number of times each gap size occurs. 
cb08 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the number of bits per gap size with a fixed encoding.
cb09 = False #@param {type:'boolean'}
#@markdown ### I correctly calculated the total number of bits encoded with the Huffman encoding.
cb10 = False #@param {type:'boolean'}
#@markdown ### I correctly calculated the total number of bits encoded with the fixed encoding.
cb11 = False #@param {type:'boolean'}
#@markdown ### I correctly calculated the compression ratio from this fixed encoding.
cb12 = False #@param {type:'boolean'}
#@markdown ### I correctly calculated the size of the first ten million primes encoded as 32-bit integer binary data.
cb13 = False #@param {type:'boolean'}
#@markdown ### I correctly calculated the compression ratio from the binary size.
cb14 = False #@param {type:'boolean'}
#@markdown ### I correctly calculated the size of the first ten million primes encoded as ASCII data.
cb15 = False #@param {type:'boolean'}
#@markdown ### I correctly calculated the compression ratio from the ASCII size (just the primes, nothing else).
cb16 = False #@param {type:'boolean'}

## DO NOT CHANGE ANYTHING IN THE NEXT CODE CELL!!
### Delete this cell and the following ones before submitting your work.

In [15]:
points_for_what_I_did = [5]*3
points_for_my_report = [8]*5
points_for_my_calculations = [5]*9
points = points_for_what_I_did + points_for_my_report + points_for_my_calculations
# cb is short for checkbox
total = sum(map(lambda n, p: p if eval(f'cb{n:02}') else 0,
                range(len(points)), points))             
total

60

# For graders

In [ ]:
#@markdown ---
number_of_MIs_found = 0 #@param {type: 'slider', min: 0, max: 5}
#@markdown ---
